<a href="https://colab.research.google.com/github/kpamungkas/DMN_FoG-ED/blob/main/ED_3DCNN_Final_DMN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Freezing of Gait-trained 3D Convolutional Neural Network on DBS-induced Executive Dysfunction Classification

# Setting Up

In [ ]:
# Setting Up
# Source : https://keras.io/examples/vision/3D_image_classification/
import os
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

# Load Data

In [ ]:
# Loading nifti data and preprocessing
import nibabel as nib
from scipy import ndimage

def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan

def process_scan(path):
    """Read volume"""
    # Read scan
    volume = read_nifti_file(path)
    return volume

In [ ]:
"""Load Dataset"""
# Folder "Worsening" consist of RSFCMaps of subjects who showed worsening in ED
# due to DBS stimulation.
trainingset_worsen_paths = [
    os.path.join("/content/drive/MyDrive/Master Thesis/ED-DMN/Worsening", x)
    for x in os.listdir("/content/drive/MyDrive/Master Thesis/ED-DMN/Worsening")
]

# Folder "Stay" consist of RSFCMaps of subjects who showed no change in ED
# due to DBS stimulation.
trainingset_no_paths = [
    os.path.join("/content/drive/MyDrive/Master Thesis/ED-DMN/Stay", x)
    for x in os.listdir("/content/drive/MyDrive/Master Thesis/ED-DMN/Stay")
]

# Folder "Improved" consist of RSFCMaps of subjects who showed improvement in ED
# due to DBS stimulation.
trainingset_improve_paths = [
    os.path.join("/content/drive/MyDrive/Master Thesis/ED-DMN/Improved", x)
    for x in os.listdir("/content/drive/MyDrive/Master Thesis/ED-DMN/Improved")
]

print("RSFCMaps with worsening ED: " + str(len(trainingset_worsen_paths)))
print("RSFCMaps with no change ED: " + str(len(trainingset_no_paths)))
print("RSFCMaps with improved ED: " + str(len(trainingset_improve_paths)))

RSFCMaps with worsening ED: 1
RSFCMaps with no change ED: 16
RSFCMaps with improved ED: 3


# Split Dataset

In [ ]:
"""Read datasets and split to training and test data"""

# Read RSFCMaps from the trainingset and store them in a 4D array according to ED change.
worsen_maps = np.array([process_scan(path) for path in trainingset_worsen_paths])
nochange_maps = np.array([process_scan(path) for path in trainingset_no_paths])
improve_maps = np.array([process_scan(path) for path in trainingset_improve_paths])

# Assign 0 for worsen_maps, 1 for nochange_maps, and 2 for improve_maps.
worsen_labels = np.array([0 for _ in range(len(worsen_maps))])
nochange_labels = np.array([1 for _ in range(len(nochange_maps))])
improve_labels = np.array([2 for _ in range(len(improve_maps))])

# Join ED data
x_ED = np.concatenate((worsen_maps, nochange_maps, improve_maps), axis=0)
y_ED = np.concatenate((worsen_labels, nochange_labels, improve_labels), axis=0)
print(
    "Number of samples in cohort 3 are %d."
    % (x_ED.shape[0])
)


Number of samples in cohort 3 are 20.


In [ ]:
""" Build a 3D CNN model"""
# tuned
# https://keras.io/examples/vision/3D_image_classification/

# Initialising the CNN
cnn = tf.keras.models.Sequential()

# 1st convolutional layer +  relu activation + pool + normalization
cnn.add(layers.Conv3D(filters=8, kernel_size=(3,3,3),  activation='relu', kernel_initializer= 'glorot_uniform', input_shape=[91, 109, 91, 1]))
cnn.add(layers.MaxPool3D((2,2,2)))
cnn.add(layers.BatchNormalization())

# 2nd convolutional layer + relu activation + pool + normalization
cnn.add(layers.Conv3D(filters=32, kernel_size=(3,3,3), activation='relu', kernel_initializer= 'glorot_uniform'))
cnn.add(layers.MaxPool3D((2,2,2)))
cnn.add(layers.BatchNormalization())

# 3rd convolutional layer + relu activation + pool + normalization
cnn.add(layers.Conv3D(filters= 64, kernel_size=(3,3,3),  activation='relu', kernel_initializer='glorot_uniform'))
cnn.add(layers.MaxPool3D((2,2,2)))
cnn.add(layers.BatchNormalization())

# 4th convolutional layer + relu activation + pool + normalization
cnn.add(layers.Conv3D(filters = 64, kernel_size=(3,3,3), activation = 'relu', kernel_initializer='glorot_uniform'))
cnn.add(layers.MaxPool3D((2,2,2)))
cnn.add(layers.BatchNormalization())

# 5th global average + pool + dropout
cnn.add(layers.GlobalAveragePooling3D())
cnn.add(layers.Dense(units = 32, activation = 'relu'))
cnn.add(layers.Dropout(0.5))

# Output Layer
#cnn2.add(layers.Flatten())
cnn.add(layers.Dense(units= 3, activation = 'softmax'))

cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 89, 107, 89, 8)    224       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 44, 53, 44, 8)     0         
_________________________________________________________________
batch_normalization (BatchNo (None, 44, 53, 44, 8)     32        
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 42, 51, 42, 32)    6944      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 21, 25, 21, 32)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 21, 25, 21, 32)    128       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 19, 23, 19, 64)    5

In [ ]:
""" Load weights from FoG-trained 3D CNN Model """

cnn.load_weights("final_dmn_model-2.h5")

In [ ]:
# Compile model
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    0.0001, decay_steps=100000, decay_rate=0.96, staircase=True
)

cnn.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc"],
)
print("Created model and loaded weights from file")

Created model and loaded weights from file


In [ ]:
""" Transform Dataset """

def coh3_preprocessing(volume, label):
    """Add one dimension to each RSFCMaps in the test set."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label

# Define data loaders
coh3_loader = tf.data.Dataset.from_tensor_slices((x_ED, y_ED))

coh3 = (
    coh3_loader
    .map(coh3_preprocessing)
    .batch(1)
)


In [ ]:
coh3_accuracy = cnn.evaluate(coh3, verbose = 0)
print("Cohort 3", "%s: %.2f%%" % (cnn.metrics_names[1], coh3_accuracy[1]*100))

Cohort 3 acc: 15.00%


In [ ]:
""" Create Confusion Matrix """

from sklearn import metrics

y_EDpred = cnn.predict_classes(coh3)

print(metrics.confusion_matrix(y_ED, y_EDpred))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[ 0  0  1]
 [ 0  0 16]
 [ 0  0  3]]


Balanced Accuracy = (00/11 + 0/6 + 3/3) /3 = 33 %